# Isomantics - SVD

### Importing Libraries

In [ ]:
! pip install -r requirements.txt
! pip install bottleneck

In [6]:
import utilities

### Main Function

In [ ]:
if __name__ == '__main__':
    # Manually set list of translations (embedding, lg1, lg2)
    
    svds = ['s','s1']
    languages = ['en','ru','de','es','fr','it', 'zh-CN']
    stats = ['min','max','mean','median','std','fro','acc']
    
    translations=[]
    
    for lang1 in languages:
        for lang2 in languages:
            translations.append(('fasttext_top',lang1, lang2))
    
    svd = {}
    svd1 = {}
    
    for translation in translations:
        embedding, lg1, lg2 = translation
        # Vocab/Vectors/Dicts
        lg1_vocab, lg1_vectors, lg2_vocab, lg2_vectors = \
            pickle_rw((lg1 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg1 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      write=False)
        lg1_dict = make_dict(lg1_vocab, lg1_vectors)
        lg2_dict = make_dict(lg2_vocab, lg2_vectors)

        print('Translation: '+lg1+'->'+lg2+'\n')

        # Train/Test Vocab/Vectors
        vocab_train, vocab_test = vocab_train_test(embedding, lg1, lg2, lg1_vocab)
        X_train, X_test, y_train, y_test = vectors_train_test(vocab_train,
                                                              vocab_test,lg1_dict,lg2_dict)
 
        
        # Fit tranlation matrix to training data
        model, history, T, tf,I, M, fro = translation_matrix(X_train, y_train)
        
        results_df = translation_results(X_test, y_test, vocab_test, T,
                                         lg2_vectors, lg2_vocab)
        acc = T_norm_EDA(results_df)
        
        U,s,Vh = SVD(T)
        
        s1 = log(s)
    
        
        for stat in stats:
            svd[stat,translation[1],translation[2]] = stat_calc(stat, s, fro, acc)
            svd1[stat,translation[1],translation[2]] = stat_calc(stat, s1, fro, acc)
            
        
    #Exporting DataFrames for SVD Heatmaps
    
    s_df = make_df(languages,languages)
    s1_df = make_df(languages,languages)
    

    for stat in stats:
        for lang1 in languages:
            for lang2 in languages:
                s_df.set_value(lang1,lang2,svd[stat,lang1,lang2])
                s1_df.set_value(lang1,lang2,svd1[stat,lang1,lang2])

        s_df.to_csv('../HeatmapData/T/s_{}.csv'.format(stat),columns = languages)
        s1_df.to_csv('../HeatmapData/T/s1_{}.csv'.format(stat),columns = languages)

    



In [1]:
s_df

NameError: name 's_df' is not defined